## Clean OPM Data
Clean Oxford Policy Management (OPM) survey data. Create Household Level dataframe with  relevant socioeconomic variables __This code is written in R__

In [1]:
# Load packages
library(tidyverse)
library(haven)

# Set up sagemaker
library(reticulate)
sagemaker <- import('sagemaker')
session <- sagemaker$Session()

# s3 bucket name
s3_bucket = "worldbank-pakistan-data"

Warning message:
“Your system is mis-configured: ‘/etc/localtime’ is not a symlink”
Warning message:
“It is strongly recommended to set envionment variable TZ to ‘Etc/UCT’ (or equivalent)”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.0     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
opm <- read_dta(session$read_s3_file(s3_bucket, 'OPM/RawData - Deidentified/bisp_combined_plist.dta'))

ERROR: Error in py_call_impl(callable, dots$args, dots$keywords): UnicodeDecodeError: 'utf-8' codec can't decode byte 0xba in position 70: invalid start byte

Detailed traceback: 
  File "/home/ec2-user/anaconda3/envs/R/lib/python3.9/site-packages/sagemaker/session.py", line 335, in read_s3_file
    return s3_object["Body"].read().decode("utf-8")



In [3]:
head(opm)

ERROR: Error in head(opm): object 'opm' not found


In [4]:
opm1 <- session$read_s3_file(s3_bucket, 'OPM/RawData - Deidentified/bisp_combined_plist.dta')

ERROR: Error in py_call_impl(callable, dots$args, dots$keywords): UnicodeDecodeError: 'utf-8' codec can't decode byte 0xba in position 70: invalid start byte

Detailed traceback: 
  File "/home/ec2-user/anaconda3/envs/R/lib/python3.9/site-packages/sagemaker/session.py", line 335, in read_s3_file
    return s3_object["Body"].read().decode("utf-8")



In [5]:
opm1

ERROR: Error in eval(expr, envir, enclos): object 'opm1' not found
